In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
#Importing Another Bank Transaction Information 
table_nm = 'DLAB_INTERN_TR_IFO_TMP' 
names = ['iqr_dt','cus_no','act_no','mkt_gb','pdt_gb','iem_cd','iem_krl_anm','bnc_qty','tot_aet_amt'] 

file_path = f'/home/sbx/data/DATA0113/DATA/{table_nm}.csv' 
df_trd = pd.read_csv(file_path,  names = names) 
df_trd.shape

FileNotFoundError: [Errno 2] File b'/home/sbx/data/DATA0113/DATA/DLAB_INTERN_TR_IFO_TMP.csv' does not exist: b'/home/sbx/data/DATA0113/DATA/DLAB_INTERN_TR_IFO_TMP.csv'

In [3]:
#Importing Bank Customer Information
table_nm = 'DLAB_HT_L1_CUS_IFO_TMP' 
names = ['cus_no','sex_dit_cd','cus_age','zip_ctp_cd', 'tco_cus_grd_cd', 'ivs_icn_cd'] 

file_path = f'/home/sbx/data/DATA0113/DATA/{table_nm}.csv' 
df_cus = pd.read_csv(file_path,  names = names) 
df_cus.head()

,cus_no,sex_dit_cd,cus_age,zip_ctp_cd,tco_cus_grd_cd,ivs_icn_cd
0,103139301,2,29,44,05,02
1,102997409,2,27,11,09,04
2,103481357,2,30,31,09,-
3,104565295,2,25,28,09,03
4,102932829,2,28,11,05,03


In [4]:
# Importing Bank Account Information
table_nm = 'DLAB_HT_L1_ACT_IFO_TMP' 
names = ['act_no','cus_no','act_opn_ym'] 

file_path = f'/home/sbx/data/DATA0113/DATA/{table_nm}.csv' 
df_act = pd.read_csv(file_path,  names = names) 
df_act.head()

,act_no,cus_no,act_opn_ym
0,20403262584,7367714,202102
1,20404262584,7367714,202102
2,20401262584,7367714,202102
3,9088211594,7367714,200511
4,20402262584,7367714,202102


In [16]:
# Making and cleaning data set to cluster
df_bal = df_trd.copy()[['iqr_dt', 'cus_no', 'tot_aet_amt']].groupby(['iqr_dt', 'cus_no']).sum()
df_bal = df_bal.reset_index()

df_bal_kor = df_trd[df_trd['mkt_gb'] == '국내'].copy()[['iqr_dt', 'cus_no', 'tot_aet_amt']].groupby(['iqr_dt', 'cus_no']).sum()
df_bal_kor = df_bal_kor.reset_index()

df_bal_ovs = df_trd[df_trd['mkt_gb'] == '해외'].copy()[['iqr_dt', 'cus_no', 'tot_aet_amt']].groupby(['iqr_dt', 'cus_no']).sum()
df_bal_ovs = df_bal_ovs.reset_index()

df_bal_tot = pd.merge(left=df_bal, 
                      right=df_bal_kor[['iqr_dt','cus_no', 'tot_aet_amt']], 
                      on=['cus_no', 'iqr_dt'], 
                      suffixes=('', '_kr'), 
                      how='left')
df_bal_tot = pd.merge(left=df_bal_tot, 
                      right=df_bal_ovs[['iqr_dt','cus_no', 'tot_aet_amt']], 
                      on=['cus_no', 'iqr_dt'], 
                      suffixes=('', '_ov'), 
                      how='left')

df_bal_tot[['tot_aet_amt_kr', 'tot_aet_amt_ov']] = df_bal_tot[['tot_aet_amt_kr', 'tot_aet_amt_ov']].fillna(0)

bank_cus = pd.merge(left=df_cus, right = df_bal_tot, on='cus_no', suffixes=('',''))
bank_cus['ovs_ratio'] = bank_cus['tot_aet_amt_ov']/bank_cus['tot_aet_amt']
bank_cus.head()


,cus_no,sex_dit_cd,cus_age,zip_ctp_cd,tco_cus_grd_cd,ivs_icn_cd,iqr_dt,tot_aet_amt,tot_aet_amt_kr,tot_aet_amt_ov,ovs_ratio
0,103139301,2,29,44,05,02,20210101,7402600,7402600.0,0.0,0.000000
1,103139301,2,29,44,05,02,20210102,8757051,7402600.0,1354451.0,0.154670
2,103139301,2,29,44,05,02,20210103,8757051,7402600.0,1354451.0,0.154670
3,103139301,2,29,44,05,02,20210104,9008351,7653900.0,1354451.0,0.150355
4,103139301,2,29,44,05,02,20210105,10911897,9561380.0,1350517.0,0.123766


In [23]:
# Making ranges for bank_cus

bank_cus_copy = bank_cus.copy()

r = np.array([0, 24, 39, 54, 80])
bank_cus_copy['cus_age'] = pd.cut(bank_cus_copy['cus_age'], r)
bank_cus_copy.head()

bank_cus_copy = bank_cus_copy.drop(columns=['cus_no', 'ivs_icn_cd', 'iqr_dt', 'tot_aet_amt_kr', 'tot_aet_amt_ov'])
bank_cus_copy.head()

,sex_dit_cd,cus_age,zip_ctp_cd,tco_cus_grd_cd,tot_aet_amt,ovs_ratio
0,2,"(24, 39]",44,05,7402600,0.000000
1,2,"(24, 39]",44,05,8757051,0.154670
2,2,"(24, 39]",44,05,8757051,0.154670
3,2,"(24, 39]",44,05,9008351,0.150355
4,2,"(24, 39]",44,05,10911897,0.123766
